In [18]:
# (Προαιρετικά) αν δεν φορτώνει σωστά τα widgets:
# !pip install -q ipywidgets plotly
from google.colab import output
output.enable_custom_widget_manager()

import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output

def compute_profit_only(x, a, b_vec, c_vec, K, call_premiums, put_premiums, S0):
    stock_payoff = a * x
    call_intrinsic = np.maximum(x[:, None] - K[None, :], 0.0)
    calls_payoff = np.dot(call_intrinsic, b_vec)
    put_intrinsic = np.maximum(K[None, :] - x[:, None], 0.0)
    puts_payoff = np.dot(put_intrinsic, c_vec)
    total_payoff = stock_payoff + calls_payoff + puts_payoff
    initial_cost = a * S0 + np.dot(b_vec, call_premiums) + np.dot(c_vec, put_premiums)
    profit = total_payoff - initial_cost
    return profit, initial_cost

def find_break_even(x, profit):
    signs = np.sign(profit)
    idx = np.where(np.diff(signs) != 0)[0]
    bes = []
    for i in idx:
        x0, x1 = x[i], x[i+1]
        y0, y1 = profit[i], profit[i+1]
        if y1 == y0:
            be = x0
        else:
            be = x0 - y0 * (x1 - x0) / (y1 - y0)
        bes.append(be)
    return np.array(bes)

# Widgets - ΌΛΑ draggable sliders
a_slider = widgets.IntSlider(value=1, min=-100, max=100, step=1, description="Shares a", layout=widgets.Layout(width="340px"))
S0_slider = widgets.FloatSlider(value=100.0, min=1.0, max=300.0, step=1.0, description="S0", layout=widgets.Layout(width="340px"))

# Προσθήκη sliders για το εύρος του γραφήματος
x_min_slider = widgets.FloatSlider(value=50.0, min=0.0, max=300.0, step=1.0, description="X Min", layout=widgets.Layout(width="340px"))
x_max_slider = widgets.FloatSlider(value=150.0, min=10.0, max=500.0, step=1.0, description="X Max", layout=widgets.Layout(width="340px"))

# b_i and c_i as draggable sliders (IntSlider για ακέραιες τιμές)
b_sliders = [widgets.IntSlider(value=0, min=-5, max=5, step=1, description=f"b{i+1}", layout=widgets.Layout(width="340px")) for i in range(5)]
c_sliders = [widgets.IntSlider(value=0, min=-5, max=5, step=1, description=f"c{i+1}", layout=widgets.Layout(width="340px")) for i in range(5)]

# Strike prices - draggable sliders
K_sliders = [widgets.FloatSlider(value=v, min=10, max=300, step=0.5, description=f"K{i+1}", layout=widgets.Layout(width="340px")) for i, v in enumerate([90,95,100,105,110])]

# Premiums - draggable sliders
call_p_sliders = [widgets.FloatSlider(value=2.0, min=0.0, max=30.0, step=0.1, description=f"Call prem {i+1}", layout=widgets.Layout(width="340px")) for i in range(5)]
put_p_sliders = [widgets.FloatSlider(value=2.0, min=0.0, max=30.0, step=0.1, description=f"Put prem {i+1}", layout=widgets.Layout(width="340px")) for i in range(5)]

# Button and outputs
plot_button = widgets.Button(description="Plot Profit", button_style="primary", layout=widgets.Layout(width="220px", height="45px"))
fig_out = widgets.Output()
summary = widgets.HTML()

def on_plot(_):
    with fig_out:
        clear_output()
        a = a_slider.value
        S0 = S0_slider.value
        b_vec = np.array([b.value for b in b_sliders], dtype=float)
        c_vec = np.array([c.value for c in c_sliders], dtype=float)
        K = np.array([k.value for k in K_sliders], dtype=float)
        call_premiums = np.array([cp.value for cp in call_p_sliders], dtype=float)
        put_premiums = np.array([pp.value for pp in put_p_sliders], dtype=float)

        # Χρήση των νέων sliders για το εύρος
        x_min = x_min_slider.value
        x_max = x_max_slider.value

        # Εξασφάλιση ότι x_min < x_max
        if x_min >= x_max:
            x_min = max(0.0, x_max - 10)

        x = np.linspace(x_min, x_max, 800)

        profit, initial_cost = compute_profit_only(x, a, b_vec, c_vec, K, call_premiums, put_premiums, S0)

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=x, y=profit, mode="lines", name="Net Profit", line=dict(width=3, color="royalblue")))
        fig.add_trace(go.Scatter(x=[x_min, x_max], y=[0, 0], mode="lines", name="Zero", line=dict(dash="dash", color="black")))

        bes = find_break_even(x, profit)
        if bes.size > 0:
            fig.add_trace(
                go.Scatter(
                    x=bes,
                    y=np.zeros_like(bes),
                    mode="markers+text",
                    name="Break-even",
                    marker=dict(symbol="diamond", size=10, color="red", line=dict(width=2, color='DarkSlateGrey')),
                    text=[f"{b:.2f}" for b in bes],
                    textposition="top center",
                )
            )

        fig.update_layout(
            title=f"Profit Profile (net of cost), initial cost = {initial_cost:.2f}",
            xaxis_title="Underlying Price at Expiration",
            yaxis_title="Profit",
            template="simple_white",
            height=450,
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        )
        display(fig)

        # Υπολογισμός πρόσθετων στατιστικών
        max_profit = np.max(profit)
        max_loss = np.min(profit)

        summary.value = f"""
        <div style="font-family: monospace; background-color: #f0f0f0; padding: 10px; border-radius: 5px;">
        <b>Strategy Summary:</b><br/>
        Initial Cost: {initial_cost:.2f} | Max Profit: {max_profit:.2f} | Max Loss: {max_loss:.2f}<br/>
        Break-even points: {[f"{b:.2f}" for b in bes] if bes.size > 0 else "None"}<br/><br/>
        <b>Parameters:</b><br/>
        a (shares): {a} | S0: {S0:.2f}<br/>
        X Range: [{x_min:.1f}, {x_max:.1f}]<br/>
        Call quantities: {b_vec.tolist()}<br/>
        Put quantities: {c_vec.tolist()}<br/>
        Strikes: {np.round(K,2).tolist()}<br/>
        Call premiums: {np.round(call_premiums,2).tolist()}<br/>
        Put premiums: {np.round(put_premiums,2).tolist()}
        </div>
        """

plot_button.on_click(on_plot)

# Reset Button
reset_button = widgets.Button(description="Reset All", button_style="warning", layout=widgets.Layout(width="100px", height="45px"))

def on_reset(_):
    a_slider.value = 1
    S0_slider.value = 100.0
    x_min_slider.value = 50.0
    x_max_slider.value = 150.0
    for i, b in enumerate(b_sliders):
        b.value = 0
    for i, c in enumerate(c_sliders):
        c.value = 0
    for i, k in enumerate([90, 95, 100, 105, 110]):
        K_sliders[i].value = k
    for cp in call_p_sliders:
        cp.value = 2.0
    for pp in put_p_sliders:
        pp.value = 2.0
    on_plot(None)

reset_button.on_click(on_reset)

# Layout
left = widgets.VBox(
    [widgets.HTML("<b>Underlying & Range</b>"), a_slider, S0_slider, x_min_slider, x_max_slider] + K_sliders,
    layout=widgets.Layout(gap="6px"),
)
middle = widgets.VBox(
    [widgets.HTML("<b>Call quantities b_i</b>")] + b_sliders + [widgets.HTML("<b>Call premiums</b>")] + call_p_sliders,
    layout=widgets.Layout(gap="6px"),
)
right = widgets.VBox(
    [widgets.HTML("<b>Put quantities c_i</b>")] + c_sliders + [widgets.HTML("<b>Put premiums</b>")] + put_p_sliders,
    layout=widgets.Layout(gap="6px"),
)

controls = widgets.HBox([left, middle, right], layout=widgets.Layout(gap="30px"))
buttons = widgets.HBox([plot_button, reset_button])

display(controls)
display(buttons)
display(fig_out)
display(summary)

# Initial draw
on_plot(None)

Output()

HTML(value='')